In [25]:
from Code.agents.MLAgent.get_game_data import load_data
from Code.agents.smart_agent import SmartAgent


import numpy as np

X1, y1 = load_data("10kgames_109715moves_MiniMax_Smart.npz")
X2, y2 = load_data("6kgames_65000moves_MiniMax_Smart.npz")
X3, y3 = load_data("6kgames_66000moves_MiniMax_Smart.npz")
X4, y4 = load_data("5kgames_55000moves_MiniMax_Smart.npz")
X5, y5 = load_data("5kgames_MiniMax_Smart_v2.npz")

combined_X = np.concatenate((X1, X2, X3, X4, X5), axis=0)
combined_y = np.concatenate((y1, y2, y3, y4, y5), axis=0)
print(f"1 - X: {X1.shape}, y: {y1.shape}")
print(f"Combined dataset shapes - X: {combined_X.shape}, y: {combined_y.shape}")

Data loaded from 10kgames_109715moves_MiniMax_Smart.npz
Data loaded from 6kgames_65000moves_MiniMax_Smart.npz
Data loaded from 6kgames_66000moves_MiniMax_Smart.npz
Data loaded from 5kgames_55000moves_MiniMax_Smart.npz
Data loaded from 5kgames_MiniMax_Smart_v2.npz
1 - X: (109715, 6, 7, 3), y: (109715, 7)
Combined dataset shapes - X: (351802, 6, 7, 3), y: (351802, 7)


In [22]:
from Code.agents.MLAgent.model import build_connect4_cnn, train_model

model = build_connect4_cnn()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ conv2d_15 (Conv2D)              │ (None, 6, 7, 64)          │      1,792 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ conv2d_16 (Conv2D)              │ (None, 6, 7, 128)         │     73,856 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ conv2d_17 (Conv2D)              │ (None, 6, 7, 128)         │    147,584 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ flatten_5 (Flatten)             │ (None, 5376)              │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_15 (Dense)                │ (None, 256)               │  1,376,512 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout_10 (Dropout)            │ (None, 256)               │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_16 (Dense)                │ (None, 128)               │     32,896 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout_11 (Dropout)            │ (None, 128)               │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_17 (Dense)                │ (None, 7)                 │        903 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 1,633,543 (6.23 MB)

 Trainable params: 1,633,543 (6.23 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
from keras.src.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
model_checkpoint = ModelCheckpoint(
    filepath='best_win_block_model.keras',
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    mode='min',
    verbose=1,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=0.00001,
    mode='min',
    verbose=1
)

In [24]:
train_model(
        model=model,
        X_train=combined_X,
        y_train=combined_y,
        epochs=15,
        batch_size=64,
        validation_split=0.2,

callbacks=[model_checkpoint, early_stopping, reduce_lr])

Start trainign fro 15 epochs.
Epoch 1/15
4398/4398 ━━━━━━━━━━━━━━━━━━━━ 91s 21ms/step - accuracy: 0.5629 - loss: 1.1548 - val_accuracy: 0.7741 - val_loss: 0.6217
Epoch 2/15
4398/4398 ━━━━━━━━━━━━━━━━━━━━ 104s 24ms/step - accuracy: 0.7816 - loss: 0.6079 - val_accuracy: 0.8090 - val_loss: 0.5211
Epoch 3/15
4337/4398 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.8160 - loss: 0.5098

KeyboardInterrupt: 

In [14]:
from Code.agents.MLAgent.model import save_trained_model
save_trained_model(model,"2ndmodel.keras")

Model successfully saved to: 1stmodel.keras


In [48]:
from Code.environment.constants import SYMBOL_PLAYER_TWO,SYMBOL_PLAYER_ONE
from Code.agents.MLAgent.MLAgent import MlAgent

player1 = MlAgent(SYMBOL_PLAYER_ONE, SYMBOL_PLAYER_TWO, "2ndmodel.keras")
player2 = SmartAgent(SYMBOL_PLAYER_TWO,SYMBOL_PLAYER_ONE)

Successfully loaded model from1stmodel.keras
ML Agent '●' initialized with model: 1stmodel.keras


In [53]:
from Code.environment.game import play_console_game
play_console_game(player1,player2)

 Welcome to Connect 4

------------------------------
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
   ┃     ┃     ┃     ┃     ┃     ┃   
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
   ┃     ┃     ┃     ┃     ┃     ┃   
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
   ┃     ┃     ┃     ┃     ┃     ┃   
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
   ┃     ┃     ┃     ┃     ┃     ┃   
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
   ┃     ┃     ┃     ┃     ┃     ┃   
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
   ┃     ┃     ┃     ┃     ┃     ┃   
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
------------------------------
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
   ┃     ┃     ┃     ┃     ┃     ┃   
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
   ┃     ┃     ┃     ┃     ┃     ┃   
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
   ┃     ┃     ┃     ┃     ┃     ┃   
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
   ┃     ┃     ┃     ┃     ┃     ┃   
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
   ┃     ┃     ┃  